In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexSets

In [ ]:
gcs = GraphOfConvexSets()

n = 3
for i in range(n):
    for j in range(n):
        v = gcs.add_vertex(f"v{(i,j)}")
        x = v.add_variable(2)
        c = np.array([i, j])
        v.add_constraint(cp.norm(x - c, 2) <= .25)

In [ ]:
for tail in gcs.vertices:
    for head in gcs.vertices:
        if tail != head:
            edge = gcs.add_edge(tail, head)
            edge.add_cost(cp.norm(head.variables[0] - tail.variables[0], 2))

In [ ]:
from itertools import combinations

ilp_constraints = []
yv = gcs.vertex_binaries()
ye = gcs.edge_binaries()

for i, vertex in enumerate(gcs.vertices):
    ilp_constraints.append(yv[i] == 1)
    inc_edges = gcs.incoming_indices(vertex)
    if i == 0:
        ilp_constraints.append(sum(ye[inc_edges]) == 0)
    else:
        ilp_constraints.append(sum(ye[inc_edges]) == 1)
        
for r in range(2, gcs.num_vertices()):
    for vertices in combinations(gcs.vertices[1:], r):
        inc_edges = gcs.incoming_indices(vertices)
        ilp_constraints.append(sum(ye[inc_edges]) >= 1)
        
for k in range(gcs.num_edges()):
    ilp_constraints.append(ye[k] >= 0)
    ilp_constraints.append(ye[k] <= 1)

In [ ]:
prob = gcs.solve_from_ilp(ilp_constraints)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.gca().set_aspect('equal')
plt.axis('off')
gcs.plot_2d()
gcs.plot_subgraph_2d()
# plt.savefig('spanning_tree.pdf')